In [4]:
import gensim.downloader as api
from collections import Counter
import torch
from torch import nn
import numpy as np
from itertools import chain
import random
from nltk.corpus import wordnet
from nltk.corpus import wordnet as wn
from num2words import num2words

POS_LIST = ['n','v','a','s','r']

print('loading word2vec')
wv = api.load('word2vec-google-news-300')
print('loading complete')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('using device',device)

def get_synonyms_antonyms(word):
    
    synsets = wordnet.synsets(word)
    if not synsets:
        return None,None
    
    synonyms = set()
    antonyms = set()

    for syn in synsets:
        for l in syn.lemmas():
            synonym = l.name()
            if synonym.isalnum():
                synonyms.add(l.name().lower())
            if l.antonyms():
                antonym = l.antonyms()[0].name()
                if antonym.isalnum():
                    antonyms.add(antonym.lower())
    synonyms.discard(word)
    return list(synonyms), list(antonyms)
    
def get_pos(word):
    total = 0
    
    out = torch.tensor(np.zeros(len(POS_LIST)),dtype=torch.float)
    synsets = wordnet.synsets(word)
    if not synsets:
        return None
    pos = Counter()
    for syn in synsets:
        pos[syn.pos()]+=1
        total += 1
    for i,a_pos in enumerate(POS_LIST):
        out[i] = pos[a_pos]/total
    return out

def get_hypernyms_hyponyms(word):
    synsets = wordnet.synsets(word)
    if not synsets:
        return None,None
    
    hypernyms,hyponyms = set(),set()
    for syn in synsets:
        for hyper in syn.hypernyms():
            candidate = hyper.name().split('.')[0]
            if candidate.isalnum():
                hypernyms.add(candidate.lower())
        
        for hypo in syn.hyponyms():
            candidate = hypo.name().split('.')[0]
            if candidate.isalnum():
                hyponyms.add(candidate.lower())
                
    return list(hypernyms),list(hyponyms)

def get_word_data(word):
    # return synonyms, antonyms, hypernyms, hyponyms, pos vector
    aux_vocab = set()
    synonyms, antonyms = get_synonyms_antonyms(word)
    hypernyms,hyponyms = get_hypernyms_hyponyms(word)
    pos_vector = get_pos(word)
    
    for word in chain(synonyms or [],antonyms or [],hypernyms or [],hyponyms or []):
        aux_vocab.add(word)
    return synonyms, antonyms, hypernyms,hyponyms,pos_vector, aux_vocab

def enrich_with_aux_vocab(w2i,i2w,i2v,i2data, aux_vocab_total,vocab):
    i = len(i2v)
    for w in aux_vocab_total:
        if not w in vocab:
            w2i[w] = i
            i += 1
            if w in wv:
                i2v.append(wv[w])
            else:
                i2v.append(torch.tensor(np.zeros(NUM_INPUT),dtype=torch.float))
            i2data.append(None)
    for word in aux_vocab_total:
        vocab.add(word)


def get_vector_vocab(vocab):
    w2i = dict()
    i2w = list(vocab)
    i2v = list()
    i2data = list()
    aux_vocab_total = set()
    for i,w in enumerate(i2w):
        w2i[w] = i
        if w in wv:
            i2v.append(wv[w])
            synonyms, antonyms, hypernyms,hyponyms,pos_vector, aux_vocab = get_word_data(w)
            for aux_w in aux_vocab:
                aux_vocab_total.add(aux_w)
            i2data.append((synonyms, antonyms, hypernyms,hyponyms,pos_vector))
        else:
            i2v.append([0 for _ in range(NUM_INPUT)])
            i2data.append((None,None,None,None,None))
    input_vocab = torch.tensor(i2v,dtype=torch.float)
    enrich_with_aux_vocab(w2i,i2w,i2v,i2data, aux_vocab_total,vocab)
    return w2i,i2w,i2v,i2data,input_vocab

def w2t(word,w2i,i2t):
    if not word or not word in w2i:
        return None
    return i2t[w2i[word]]

def sample_choice(group_a,group_b,w2i,i2t):
    if np.random.random()<0.5 and group_a:
        return w2t(random.choice(group_a),w2i,i2t),torch.tensor([1],dtype=torch.float)
    elif group_b:
        return w2t(random.choice(group_b),w2i,i2t),torch.tensor([0],dtype=torch.float)
    return None,None

class EmbeddingModifierNetwork(nn.Module):
    def __init__(self):
        super(EmbeddingModifierNetwork, self).__init__()
        self.hidden = nn.Linear(NUM_INPUT,NUM_HIDDEN)
        
        self.equality_layer = nn.Linear(NUM_HIDDEN,NUM_INPUT)
        self.synonym_antonym_layer = nn.Linear(NUM_HIDDEN+NUM_INPUT,1)
        self.hypernym_hyponym_layer = nn.Linear(NUM_HIDDEN+NUM_INPUT,1)
        self.pos_layer = nn.Linear(NUM_HIDDEN,len(POS_LIST))

    def forward(self,x,syn_or_ant,hyper_or_hypo):
        x = self.hidden(x)
        
        # main output layer
        main_out = self.equality_layer(x)
        main_out = torch.sigmoid(main_out)
        
        syn_ant_out = None
        if syn_or_ant is not None:
            syn_ant_out = self.synonym_antonym_layer(torch.hstack([x,syn_or_ant]))
            syn_ant_out = torch.sigmoid(syn_ant_out)
        
        hyper_hypo_out = None
        if hyper_or_hypo is not None:
            hyper_hypo_out = self.synonym_antonym_layer(torch.hstack([x,hyper_or_hypo]))
            hyper_hypo_out = torch.sigmoid(hyper_hypo_out)
            
        pos_out = self.pos_layer(x)
        pos_out = torch.softmax(pos_out,dim=0)
        
        return main_out,syn_ant_out,hyper_hypo_out,pos_out

loading word2vec
loading complete
using device cpu


In [2]:
import pickle

with open('vocab.dill', 'rb') as file:
    vocab = pickle.load(file)

In [2]:
import torch
import dill as pickle

In [6]:
NUM_INPUT = 300

w2i,i2w,i2v,i2data,input_vocab = get_vector_vocab(vocab)

<ipython-input-4-fc6ad6a643c4>:119: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  input_vocab = torch.tensor(i2v,dtype=torch.float)


In [8]:
i2t = torch.tensor(i2v,dtype=torch.float64)

In [9]:
# serialize for num input = 300

with open('w2i.dill', 'wb') as file:
    pickle.dump(w2i, file)
    
with open('i2w.dill', 'wb') as file:
    pickle.dump(i2w, file)
    
with open('i2data.dill', 'wb') as file:
    pickle.dump(i2data, file)
    
torch.save(i2t,'word2vec_embeds.pt')

In [5]:
with open('w2i.dill', 'rb') as file:
    w2i = pickle.load(file)
    
with open('i2w.dill', 'rb') as file:
    i2w = pickle.load(file)
    
with open('i2data.dill', 'rb') as file:
    i2data = pickle.load(file)
    
embeds = torch.load('word2vec_embeds.pt')

In [8]:
l

tensor([[ 0.3125, -0.2129,  0.0874,  ..., -0.0300,  0.1523,  0.0659],
        [ 0.0483,  0.1279,  0.1064,  ..., -0.1084,  0.1582, -0.0469],
        [ 0.1582,  0.2734, -0.1963,  ...,  0.1143,  0.0767, -0.0051],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0400, -0.0038,  0.0034,  ..., -0.2119,  0.2812,  0.3301]],
       dtype=torch.float64)